In [1]:
import numpy as numpy
import matplotlib.pyplot as plt 
import pandas as pd 


In [2]:
import feets.preprocess
from scipy import stats
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import feets
from ipykernel import kernelapp as app
import os



script_dir = os.path.dirname('Results/')
results_dir = os.path.join(script_dir, 'Images/')

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

    
Plastic_data = pd.read_csv('test_set_batch1.csv')

In [3]:
url = "https://raw.githubusercontent.com/MachineLearningUniandes/MANTRA/master/data/lightcurves/transient_lightcurves.csv"
url1 = 'https://raw.githubusercontent.com/MachineLearningUniandes/MANTRA/master/data/lightcurves/transient_labels.csv'
url2 = 'https://raw.githubusercontent.com/MachineLearningUniandes/MANTRA/master/data/lightcurves/transient_info.txt'
transient_lc = pd.read_csv(url)
transient_labels = pd.read_csv(url1)
transient_info = pd.read_table(url2,skiprows=1,names=['CRTS_ID' ,'RA' ,'Dec','UT_Date','Mag','CSS_images' ,'SDSS',
                                                      'Others' ,'Followed' ,'Last','LC','FC','Classification'])

transient_info.head()

,CRTS_ID,RA,Dec,UT_Date,Mag,CSS_images,SDSS,Others,Followed,Last,LC,FC,Classification
0,CSS120828:000010-215515,0.04323,-21.92087,20120828.38,17.02,1208280210014111715,no,11171,no,2016-10-22,11171,yes,Var
1,CSS100910:000025+332543,0.10280,33.42861,20100910.42,15.85,1009101320014148188,yes,14818,no,2016-11-05,14818,yes,"CV SDSS mag 20,5"
2,CSS101127:000130+050624,0.37695,5.10657,20101127.21,15.68,1011271040014129121,no,12912,yes,2016-11-06,12912,yes,"CV mag 20,0"
3,CSS160709:000132+211336,0.38474,21.22680,20160709.45,17.21,1607091210014124380,yes,12438,no,2016-11-05,12438,yes,Blazar CRATES_J0001+2113
4,CSS110926:000156+065236,0.48205,6.87665,20110926.30,19.35,1109261070014122864,no,12286,yes,2016-11-06,12286,yes,"SN 2011hl SDSS mag 15,7 2MASX_00015920"


# First attempt to the function

In [4]:
def read_lc_from_file(flpath,lc_columns,fl_columns,header_nrows):


        # Possible file extensions
        extn = ['.csv','.dat','.txt']

        for ext in extn:

            
            # ==========CSV_files=============
            if ext in flpath and ext == extn[0]:

                # Reading in the data
                light_curve=pd.read_csv(flpath,skiprows=header_nrows,names=fl_columns)
                # Selecting columns with lc data
                light_curve = light_curve.loc[:,lc_columns]            
                # Renaming the columns into standard names for astronomaly
                light_curve.columns = ['time','mag','magerr']
                
                return light_curve

            # ==========.Dat_file=============
            else:
                
                # Reading in the data
                light_curve = pd.read_table(flpath,skiprows=header_nrows,names=fl_columns,delim_whitespace=True)
                # Selecting columns with lc data
                light_curve = light_curve.loc[:,lc_columns]
                # Renaming the columns into standard names for astronomaly
                light_curve.columns = ['time','mag1','mag2']
                
                return light_curve

# Second attempt

In [5]:
def read_lc1(flpath,data_dict):
    
    '''Function to read the lc from the data
    
    Input:
    
    flpath: the location of the file
    
    data_dict: dictionary with the keys ('time','mag','magerr')
               the user provides the values corresponding to the keys
               e.g {'time':'mjd'}, were 'mjd' is the time column name in the 
               data
               
   Output:
   
   standardized pandas dataframe with lc data'''
    
    
    # Reading-in the data
    data = pd.read_csv(flpath)
        
    
    # This for column names 
    if type(data_dict['time']) == str:
    
        # Separatting the columns as per input dictionary
        time = data.loc[:,data_dict['time']]; 
        mag = data.loc[:,data_dict['mag']]; 
        error = data.loc[:,data_dict['mag_errors']]
        
    
    # This is for column indx
    else:
        
        # Separatting the columns as per input dictionary
        time = data.iloc[:,data_dict['time']]; 
        mag = data.iloc[:,data_dict['mag']]; 
        error = data.iloc[:,data_dict['mag_errors']]
    
    # Creating a new dictionary for the columns above separate data
    new_data = {'time':time,'mag':mag,'error':error}
    
    
    
    return pd.DataFrame.from_dict(new_data)

# 3rd attempt of the function

In [6]:
# I think the best is to use the column indexs
def read_lc2(flpath,data_dict):
    
    '''Function to read the lc from the data
    
    Input:
    
    flpath: the location of the file
    
    data_dict: dictionary with the keys ('time','mag','mag_err','flux','flux_err','filters')
               the user provides the values corresponding to the keys
               e.g {'time':1}, were 1 is the time column index
               
    brightness_unit: Units used to measure the brightness
                     can either be 'flux' or 'mags'
               
   Output:
   
   standardized pandas dataframe with lc data'''
    
    
    # Reading-in the data
    data = pd.read_csv(flpath)
        
    if 'mag' in data_dict.keys(): 
        
        # Separatting the columns as per input dictionary
        time = data.iloc[:,data_dict['time']]; 
        mag = data.iloc[:,data_dict['mag']]; 
        mag_error = data.iloc[:,data_dict['mag_err']]
        # Creating a new dictionary for the columns above separate data
        standard_data = {'time':time,'mag':mag,'mag_error':mag_error}
        
        # Including filters in dataframe
        if 'filters' in data_dict.keys():
            
            filters = data.iloc[:,data_dict['filters']]
            standard_data = {'time':time,'mag':mag,'mag_error':mag_error,'filters':filters}
            
            
            
        
    else:
        
        # Separatting the columns as per input dictionary
        time = data.iloc[:,data_dict['time']]; 
        flux = data.iloc[:,data_dict['flux']]; 
        flux_error = data.iloc[:,data_dict['flux_err']]
    
        standard_data = {'time':time,'flux':flux,'flux_error':flux_error}
        
        # Including filters in the dataframe
        if 'filters' in data_dict.keys():
            
            filters = data.iloc[:,data_dict['filters']]
            standard_data = {'time':time,'flux':flux,'flux_error':flux_error,'filters':filters}
            
    
    
    
    return pd.DataFrame.from_dict(standard_data)

In [7]:
read_lc2(flpath='test_set_batch1.csv',data_dict={'time':1,'flux':3,'flux_err':4,'filters':2})

,time,flux,flux_error,filters
0,59798.3205,-1.299735,1.357315,2
1,59798.3281,-2.095392,1.148654,1
2,59798.3357,-0.923794,1.763655,3
3,59798.3466,-4.009815,2.602911,4
4,59798.3576,-3.403503,5.367328,5
...,...,...,...,...
10855953,60434.0005,13.435188,1.301140,3
10855954,60434.0115,11.484802,1.979236,4
10855955,60434.0224,13.092712,4.928932,5
10855956,60435.9857,2.289347,2.553590,0


In [8]:
read_lc2(flpath=url,data_dict={'time':4,'mag':2,'mag_err':3})

,time,mag,mag_error
0,53766.089871,18.8765,0.166417
1,53990.458866,20.0519,0.281733
2,53996.286004,20.2199,0.295764
3,54385.205789,21.1192,0.495390
4,54355.282285,19.3289,0.195002
...,...,...,...
440464,56189.314816,18.1952,0.123607
440465,56189.322996,18.2097,0.124706
440466,56205.285887,18.5116,0.138816
440467,56205.292631,18.3150,0.128915
